In [ ]:
"""
1. 設定菜單 設定檔
2. 讀取安全設定
3. 設定菜單照片
4. 上傳給Line
5. 測試將自己的ID綁入設定好的菜單


"""



In [ ]:
"""
設定菜單

"""

menu ="""
{
  "size": {
    "width": 2500,
    "height": 1686
  },
  "selected": true,
  "name": "圖文選單 2",
  "chatBarText": "查看更多資訊",
  "areas": [
    {
      "bounds": {
        "x": 256,
        "y": 132,
        "width": 891,
        "height": 776
      },
      "action": {
        "type": "message",
        "text": "[::text:] 吃啥"
      }
    },
    {
      "bounds": {
        "x": 252,
        "y": 912,
        "width": 907,
        "height": 774
      },
      "action": {
        "type": "message",
        "text": "[::text:] 位置推薦"
      }
    },
    {
      "bounds": {
        "x": 1349,
        "y": 124,
        "width": 908,
        "height": 792
      },
      "action": {
        "type": "message",
        "text": "[::text:] 圖片推薦"
      }
    },
    {
      "bounds": {
        "x": 1349,
        "y": 920,
        "width": 916,
        "height": 763
      },
      "action": {
        "type": "uri",
        "uri": "tel://3345678"
      }
    }
  ]
}"""





In [ ]:
"""
讀取安全設定


"""
import json

secretfileJson = json.load(open("./line_secret_key"))

channel_access_token = secretfileJson["channel_access_token"]
secret_key = secretfileJson["secret_key"]
self_user_id = secretfileJson["self_user_id"]
rich_menu_id = secretfileJson["rich_menu_id"]

print(channel_access_token)
print(secret_key)
print(self_user_id)
print(rich_menu_id)

In [ ]:
"""
設定菜單照片
並上傳給LINE


"""
import requests
myMenu = json.loads(menu)

# Line Menu End Point
createMenuEndpoint = 'https://api.line.me/v2/bot/richmenu'

# 組合 header Bearer 後加入 channel access token
createMenuRequestHeader={'Content-Type':'application/json', 'Authorization':'Bearer %s' % channel_access_token}


#print(createMenuRequestHeader)


# 以 Post 發給 Line 註冊 Menu
# 必須要使用 Json dump 
lineCreateMenuResponse = requests.post(url=createMenuEndpoint, headers=createMenuRequestHeader, data=json.dumps(myMenu))

print(lineCreateMenuResponse)
print(lineCreateMenuResponse.text)






In [ ]:
print(secretfileJson["rich_menu_id"])
# 將取得的 richMenuId 寫入到原有的 secretfile

# 取代 原有的 MenuId
secretfileJson["rich_menu_id"] = json.loads(lineCreateMenuResponse.text)["richMenuId"]

# 寫入
# r+ 讀寫
# indent = 2 保留原有的 \n 方便閱讀
f = open("./line_secret_key","r+")
f.write(json.dumps(secretfileJson,indent=2))


In [ ]:
"""
針對指定的 Menu 上傳照片給 LINE


"""
# 先取得菜單ID
# 使用剛剛註冊好的 較為保險
uploadRichMenuId=json.loads(lineCreateMenuResponse.text)["richMenuId"]
# print(uploadRichMenuId)

#'https://api.line.me/v2/bot/richmenu/{richMenuId}/content'

# 設定Line的遠端位置
uploadMenuEndpoint='https://api.line.me/v2/bot/richmenu/%s/content' % uploadRichMenuId

# 設定 Request header
# 跟原本不同的是 content-type 為 imgae
uploadMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % channel_access_token}


# 設定上傳圖片
uploadImageFile=open("./Images/menu.jpg",'rb')

# Post 發給 Line
lineUploadMenuResponse = requests.post(url=uploadMenuEndpoint, headers=uploadMenuRequestHeader, data=uploadImageFile)

print(lineUploadMenuResponse)
print(lineUploadMenuResponse.text)




In [ ]:
"""
將自己的ID綁入測試菜單



"""
# 取得菜單 ID 
# 可以分開使用 也可以一起用
linkMenuId = uploadRichMenuId

# 組合成遠端位置

# https://api.line.me/v2/bot/user/{userId}/richmenu/{richMenuId}
linkMenuEndpoint = "https://api.line.me/v2/bot/user/%s/richmenu/%s" % (self_user_id, linkMenuId)

# 設定 header
linkMenuHeader = {'Content-Type':'image/jpeg','Authorization':'Bearer %s' % channel_access_token}

# 發送綁定請求
linkMenuResponse = requests.post(url=linkMenuEndpoint, headers=linkMenuHeader)

print(linkMenuResponse)
print(linkMenuResponse.text)





In [ ]:
'''

檢視用戶目前所綁定的菜單
    

'''
# 步驟其實一樣 只有在發送消息時
# 使用 get

# 取出id，設定Line的遠端位置
userMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu' % (self_user_id)
# print(userMenuEndpoint)

# 設定消息的基本安全憑證
userMenuRequestHeader={'Authorization':'Bearer %s' % channel_access_token}

# 發送消息告知
lineUnregisterUserMenuResponse=requests.get(userMenuEndpoint,headers=userMenuRequestHeader)
print(lineUnregisterUserMenuResponse)
print(lineUnregisterUserMenuResponse.text)





In [ ]:
"""
可以檢視 就可以刪除
    步驟一樣
    requests 時 使用 delete


"""
lineUnregisterUserMenuResponse=requests.delete(userMenuEndpoint,headers=userMenuRequestHeader)
print(lineUnregisterUserMenuResponse)
print(lineUnregisterUserMenuResponse.text)



In [ ]:
"""
檢視帳號內的所有菜單
    只有LINE遠端不同
    使用 get


"""

# 設定Line的遠端位置
listMenuEndpoint="https://api.line.me/v2/bot/richmenu/list"

lineLisrMenuResponse = requests.get(listMenuEndpoint,headers=userMenuRequestHeader)

print(lineLisrMenuResponse)
print(lineLisrMenuResponse.text)


In [ ]:
json.loads(lineLisrMenuResponse.text)